Hacker News is a social news website focusing on computer science and entrepreneurship. It has a community where users can submit articles, and other users can upvote those articles. Like other website the articles with the most upvotes make it to the front page.

This data set consists of submissions users made to Hacker News from 2006 to 2015. Developer Arnaud Drizard used the Hacker News API to scrape the data, which which can be  found in one of his GitHub repositories. https://github.com/arnauddri/hn

hn_stories is a 3000 rows that was sampled from the data randomly, and it has only has four columns:

* submission_time - When the article was submitted
* upvotes - The number of upvotes the article received
* url - The base URL of the article
* headline - The article's headline

I'll be predicting the number of upvotes the articles received, based on their headlines. 
Upvotes are an indicator of popularity, I will try to discover which types of articles tend to be the most popular it this community.

-------


In [69]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

In [70]:

# Reading the file to datframe
articals = pd.read_csv("hn_stories.csv")
articals.columns = ["submission_time", "upvotes", "url", "headline"]


In [72]:
# Exploring the data
articals.head()

,submission_time,upvotes,url,headline
0,2010-02-17T16:57:59Z,1,blog.jonasbandi.net,Software: Sadly we did adopt from the construc...
1,2014-02-04T02:36:30Z,1,blogs.wsj.com,Google’s Stock Split Means More Control for L...
2,2011-10-26T07:11:29Z,1,threatpost.com,SSL DOS attack tool released exploiting negoti...
3,2011-04-03T15:43:44Z,67,algorithm.com.au,Immutability and Blocks Lambdas and Closures
4,2013-01-13T16:49:20Z,1,winmacsofts.com,Comment optimiser la vitesse de Wordpress?


In [73]:

articals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 4 columns):
submission_time    2999 non-null object
upvotes            2999 non-null int64
url                2810 non-null object
headline           2989 non-null object
dtypes: int64(1), object(3)
memory usage: 93.8+ KB


As we can see here, we have four columns three with an object data type and one numerical.

There are some missing values, so I am going to remove those rows first

In [76]:
articals = articals.dropna()

-----
Since I am going to train a linear regression model to predicts the number of upvotes a headline would receive, I will need to convert each headline to a numerical representation.
There are several ways to do this, I will use the bag of words model where each piece of text is represented as a numerical vector.
The first step in creating a bag of words model is tokenization. I am going to split each sentence into a list of individual words on the space character

In [77]:
# Creating a list for the tokens
headline_tokens = []

# Looping through the dataframe to split headlines and add words as a list 
for row in articals['headline']:
    headline_tokens.append(row.split())
    

In [79]:
headline_tokens[:2]

[['Software:',
  'Sadly',
  'we',
  'did',
  'adopt',
  'from',
  'the',
  'construction',
  'analogy'],
 ['Google’s',
  'Stock',
  'Split',
  'Means',
  'More',
  'Control',
  'for',
  'Larry',
  'and',
  'Sergey']]

----
On my next step, I am going to remove punctuation lowercas all words.

In [81]:
# A list of punctuation to be removed from tokens
punctuation = [ "/", "-", "+", "&", "(", ")", ",", ":", ";", ".", "'", '"', "’", "?"]

# New list for the processed tokens
clean_tokens = []

# A loop to go through tokens and lower case each word and remove punctuation 
for tokens in headline_tokens:
    #list for each sentance
    tokens_list = []
    for token in tokens:
        token = token.lower()
        for p in punctuation:
            token = token.replace(p, '')
        tokens_list.append(token)
    clean_tokens.append(tokens_list)
    
clean_tokens[:2]

[['software',
  'sadly',
  'we',
  'did',
  'adopt',
  'from',
  'the',
  'construction',
  'analogy'],
 ['googles',
  'stock',
  'split',
  'means',
  'more',
  'control',
  'for',
  'larry',
  'and',
  'sergey']]

-----
Now I am going to create a dataframe for all the unique words to convert the sentences to their numerical representations.
I will only keep tokens that occured more than one time, tokens that only occurred once don't add to the model's prediction power.

In [82]:

# A list for tokens that only occured once
single_tokens = []
# List for unique tokens
unique_tokens = []

# loop through the clean tokens list to add the unique once
for tokens in clean_tokens:
    for token in tokens:
        if token not in single_tokens:
            single_tokens.append(token)
        elif token not in unique_tokens:
            unique_tokens.append(token)

# creating a dataframe for all unique tokens as columns names and intialize it with 0 and same size as the clean_tokens list
tokens_df = pd.DataFrame(0, index=np.arange(len(clean_tokens)), columns = unique_tokens)


In [83]:
tokens_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2800 entries, 0 to 2799
Columns: 2310 entries, and to disaster
dtypes: int64(2310)
memory usage: 49.4 MB


-----
I will loop through the dataframe I just created and add 1 to the tokens if it is in the list of unique tokens



In [84]:
for index, tokens in enumerate(clean_tokens):
    for token in tokens:
        if token in unique_tokens:
            tokens_df.iloc[index][token] += 1
            

In [85]:
tokens_df.iloc[:10,:20]

,and,for,as,you,is,the,split,good,how,what,,of,de,in,a,with,amazon,cloud,at,google
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,0,1,2,2,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
7,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


To further inhance my prediciton, I am going to remove words that occured more than 100 times which should remove stopwords like 'and' and 'for', which occurs almost in every headline and words that occured less than 5 times. 



In [86]:
# get the sum of each token occurrence 
word_counts = tokens_df.sum(axis = 0)

# remove tokens fewer than 5 and more than 100
tokens_df = tokens_df.loc[:, (word_counts >= 5) & (word_counts <= 100)]

In [50]:
tokens_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2800 entries, 0 to 2799
Columns: 661 entries, as to nike
dtypes: int64(661)
memory usage: 14.1 MB


we reduced tokens from 2310 to 661

I will split the data into two sets: 80% training set and 20% test set, 
and I will use the train_test_split function from sckit-learn to do that 


In [87]:

x_train, x_test, y_train, y_test = train_test_split(tokens_df, articals['upvotes'], test_size = 0.2)

In [88]:

# initializing the linear regression model
lr = LinearRegression()

# Fitting the model
lr.fit(x_train, y_train)

# Calculating predictions on the test data
predictions = lr.predict(x_test)



In [91]:

mean = np.mean(predictions)
std  = np.std(predictions)
mse = mean_squared_error(predictions, y_test)
rmse = np.sqrt(mse)

print('Mean = ', mean)
print('Srandard Deviation = ', std)
print('Mean Squared Error = ', mse)
print('Squared root of mse = ', rmse)


Mean =  10.32407204900292
Srandard Deviation =  29.084076705842442
Mean Squared Error =  1815.5326272886325
Squared root of mse =  42.60906743040303


Mean squared error is 1815 which is a fairly larg number. The mean  of upvotes is 10, and the standard deviation is 29.
The square root of mse is 42.6 which means that the average error is 46 upvotes away from the true value which higher than the standard deviation, so my predictions are off-base.
I will try a random forest model, maybe I can get better results.

In [92]:
# initializing the the model
rf = RandomForestClassifier(random_state = 1)

#fitting the model
rf.fit(x_train, y_train)

# Calculating predictions
predictions = rf.predict(x_test)

mean = np.mean(predictions)
std  = np.std(predictions)
mse = mean_squared_error(predictions, y_test)
rmse = np.sqrt(mse)

print('Mean = ', mean)
print('Srandard deviation = ', std)
print('Mean squared error = ', mse)
print('Squared root of mse = ', rmse)

Mean =  6.1553571428571425
Srandard deviation =  36.77478474076486
Mean squared error =  2353.6696428571427
Squared root of mse =  48.51463328581535


The random forest rmse is 48.5 which is more than what I got from the regression model. 
I will leave it for now and try to do some more features engineering and hyperparameter optimization in the future to get a better result. 